In [3]:
import requests
import json
from lxml import etree, html as lhtml
from bs4 import BeautifulSoup
import pandas as pd
from itertools import chain
from multiprocessing import Pool, Lock, Value
from time import sleep

In [4]:
authors = [9150,
59396,
28727,
53427,
26268,
16626,
28927,
37969,
104832]

In [5]:
mutex = Lock()
n_processed = Value('i', 0)

def func(author_id):
    all_books_id = []
    cur_page = 1
    while(1):
        for t in range(5):
                html_21 = requests.get("https://www.moscowbooks.ru/catalog/author/{0}/?page={1}".format(author_id,cur_page))
                if html_21.status_code==200:
                    break
        if html_21.status_code!=200:
            break
        film_html = html_21.text
        tree = lhtml.fromstring(film_html)
        soup = BeautifulSoup(film_html, 'html.parser')
        soup = BeautifulSoup(film_html, 'lxml')
        elms = soup.select("div.book-preview__cover a")
        for i in elms:
            all_books_id.append(i.attrs["href"])
        if not elms:
            break
        cur_page = cur_page + 1
    return all_books_id

def func_wrapper(uid):
    res = func(uid) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    all_books_id = pool.map(func_wrapper, authors)

In [6]:
all_books_id = list(chain.from_iterable(list(all_books_id)))

In [7]:
def func1(book_id, n_attempts=5):
    dict_book = {}

    for t in range(n_attempts):
        html_21 = requests.get('https://www.moscowbooks.ru{}'.format(book_id))
        if html_21.status_code == 200:
            break
    if html_21.status_code!=200:
            return {}
    book_html = html_21.text
    tree = lhtml.fromstring(book_html)
    soup = BeautifulSoup(book_html, 'html.parser')
    soup = BeautifulSoup(book_html, 'lxml')
#      код товара
    # таблица
    elms = soup.select("dt.book__details-value a")
    w1 = [i.text.strip()[:-1] for i in soup.select("dt.book__details-name")]
    w2 = [i.text.strip() for i in soup.select("dt.book__details-value")]
    dict_book['Код товара'] = w2[w1.index('Код товара')]
    # автор
    elms = soup.select("div.page-header__author a")
    author = [i.text for i in elms]
    dict_book["Автор"] = ';'.join(author)
    # название
    elms = soup.select("div.book__rating")
    elms = soup.find("span",{"class":"link-gray-light"})
    dict_book["Название"] = elms.text
    # наличие
    elms = soup.select("span.instock1")
    if not elms:
        dict_book["Наличие"] = False
    else:
        dict_book["Наличие"] = bool(elms[0].text.rfind("В наличии"))

    # описание
    el  = soup.find("div",{'class':"book__description collapsed js-book-description"})
    t1 = el.b.text
    description = el.text
    description = description.replace(t1,'').strip('\n').rstrip('\s').replace("Читать дальше…",'')
    dict_book["Описание"] = description[:description.rfind('.') + 1]
    
    # рейтинг
    div = soup.find('div',{'class':"book__rating"})
    dict_book["Рейтинг"] = div.div.attrs['data-rate']
    # обложка
    elms = soup.select("div.book__cover a")
    dict_book["Обложка"] = "https://www.moscowbooks.ru" + elms[0].attrs["href"]

    # стикеры
    stick = []
    div = soup.find('div',{'class':"book__stickers stickers stickers_lg"})
    if not div is None:
        stick = div.text.split('\n')
        if not div.span is None:
            stick.append(div.span.get('title'))
        stick2 = [i for i in stick if i is not '']
        if stick2:
            dict_book["Стикеры"] = ';'.join(stick2)

    # таблица
    dict_book.update(dict(zip(w1, w2)))
    # id
    book = book_id.split('/')
    dict_book["ID"] = book[-2]
#     цена
    div = soup.find('div',{'class':"book__price"})
    price = div.text.replace('\n','').replace('\r','').replace('\s','').strip()
    dict_book["Цена"] = price
    return dict_book


In [8]:
mutex = Lock()
n_processed = Value('i', 0)


def func_wrapper(uid):
    res = func1(uid) 
    with mutex:
        # в этом блоке можно безопасно менять общие объекты для процессов
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    res = pool.map(func_wrapper, all_books_id)

190 objects are processed...

In [9]:

df = pd.DataFrame(res)
df.sort_values(by=['ID'], inplace=True)

with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)